# Preprocessing

Imports

In [4]:
import os
import math
import json
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

Constants declared

In [5]:
DATASET_PATH = "../raw_data/genre/genres_original/"
JSON_PATH = "../raw_data/genre/data.json "
SAMPLE_RATE = 22050
DURATION = 30 # seconds per track
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

Test a file

In [ ]:
_ = '../raw_data/genre/genres_original/hiphop/hiphop.00007.wav'

In [ ]:
import IPython
IPython.display.Audio(filename= _)

## Retrieve MFCCs

Function created to walk through path, extract labels, perform STFT tranforms on audio files and store the MFCCs from those. 

If the train keyword is set to False, it will traet the dataset_path as one new music file path and perform a split if the song is more than 30 seconds or request a longer song if it is less. It stores the mfcc in a json file with the same structure as the train files with the exception that the genre and label keys are empty.

In [7]:
def save_mfcc(dataset_path, json_path, n_mfcc= 13, n_fft= 2048, hop_length= 512, num_segments= 5, train=True):
    SAMPLE_RATE = 22050
    DURATION = 30 # seconds per track
    SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION
    
    # dictionary to store data
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": []
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_nmfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    if train:
        # loop through all genres
        for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

            # ensure we are not at root level
            if dirpath is not dataset_path:
                # save the semantic label
                semantic_label = dirpath.split("/")[-1] # genre/blues => ['genre', 'blues']
                data['mapping'].append(semantic_label)
                print(f"\nProcessing {semantic_label}")

                # process files for a specific genre
                for f in filenames[:50]:
                    # load audio file
                    file_path = os.path.join(dirpath, f)
                    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

                    # process by segment, extracting mfcc and storing data
                    for s in range(num_segments):
                        start_sample = num_samples_per_segment * s 
                        finish_sample = start_sample + num_samples_per_segment

                        mfcc = librosa.feature.mfcc(signal[start_sample: finish_sample],
                                                    sr=sr,
                                                    n_fft= n_fft,
                                                    n_mfcc= n_mfcc,
                                                    hop_length = hop_length)
                        mfcc = mfcc.T

                        # store mfcc for segment if it has the expected length
                        if len(mfcc) == expected_nmfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print(f"{file_path}, segment: {s+1}")

            with open(json_path, 'w') as fp:
                json.dump(data, fp, indent=4)
    
    # for new audio file, it will need to be split into 30 sec bits
    if not train:
        signal, sr = librosa.load(dataset_path)

        start = 0
        split_data = {}
        duration = int(librosa.get_duration(signal)) # duration

        if duration < 30:
            print("Please upload a song of at least 30 seconds")

        else:
            batch_size = sr * 30  # get num of vectors per 30 secs
            length = len(signal) # total len of signal array
            num_iter = duration // 30 # model trained with 30 second clips
            # num of times to loop over len 
            for i in range(num_iter):
                # slice 30 seconds--> batch size 
                split_data['batch_'+ str(i+1)] = signal[start: start+batch_size]
                start += batch_size  #update start index

        for key, value in split_data.items():
            signal = value
            
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s 
                finish_sample = start_sample + num_samples_per_segment
                    
                mfcc = librosa.feature.mfcc(signal[start_sample: finish_sample],
                                            sr=sr,
                                            n_fft= n_fft,
                                            n_mfcc= n_mfcc,
                                            hop_length = hop_length)
                mfcc = mfcc.T
                    
                # store mfcc for segment if it has the expected length
                if len(mfcc) == expected_nmfcc_vectors_per_segment:
                    data["mfcc"].append(mfcc.tolist())
                    print(f"{key}, segment: {s+1}")
                    
        with open(json_path, 'w') as fp:
                json.dump(data, fp, indent=4)

Result stored as `.json` file below.

In [8]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


Processing blues
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 5
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 6
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 7
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 8
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 9
../raw_data/genre/genres_original/blues\blues.00000.wav, segment: 10
../raw_data/genre/genres_original/blues\blues.00001.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00001.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00001.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00001.wav, segment: 4
../raw_data/genre/genres_orig

../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 5
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 6
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 7
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 8
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 9
../raw_data/genre/genres_original/blues\blues.00012.wav, segment: 10
../raw_data/genre/genres_original/blues\blues.00013.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00013.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00013.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00013.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.0

../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 5
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 6
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 7
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 8
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 9
../raw_data/genre/genres_original/blues\blues.00024.wav, segment: 10
../raw_data/genre/genres_original/blues\blues.00025.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00025.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00025.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00025.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.0

../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 5
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 6
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 7
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 8
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 9
../raw_data/genre/genres_original/blues\blues.00036.wav, segment: 10
../raw_data/genre/genres_original/blues\blues.00037.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00037.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00037.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00037.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.0

../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 5
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 6
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 7
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 8
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 9
../raw_data/genre/genres_original/blues\blues.00048.wav, segment: 10
../raw_data/genre/genres_original/blues\blues.00049.wav, segment: 1
../raw_data/genre/genres_original/blues\blues.00049.wav, segment: 2
../raw_data/genre/genres_original/blues\blues.00049.wav, segment: 3
../raw_data/genre/genres_original/blues\blues.00049.wav, segment: 4
../raw_data/genre/genres_original/blues\blues.0

...

../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 5
../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 6
../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 7
../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 8
../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 9
../raw_data/genre/genres_original/reggae\reggae.00048.wav, segment: 10
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 1
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 2
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 3
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 4
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 5
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 6
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 7
../raw_data/genre/genres_original/reggae\reggae.00049.wav, segment: 8
../raw_data/genre/g

../raw_data/genre/genres_original/rock\rock.00010.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00010.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00010.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00010.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 1
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 2
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 4
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 5
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00011.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00012.wav, segment: 1
../raw_d

../raw_data/genre/genres_original/rock\rock.00022.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 1
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 2
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 4
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 5
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00023.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00024.wav, segment: 1
../raw_data/genre/genres_original/rock\rock.00024.wav, segment: 2
../raw_data/genre/genres_original/rock\rock.00024.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00024.wav, segment: 4
../raw_d

../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 4
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 5
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00035.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 1
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 2
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 4
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 5
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00036.wav, segment: 7
../raw_da

../raw_data/genre/genres_original/rock\rock.00047.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00047.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00047.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00047.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00047.wav, segment: 10
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 1
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 2
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 3
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 4
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 5
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 6
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 7
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 8
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 9
../raw_data/genre/genres_original/rock\rock.00048.wav, segment: 10
../raw_d

## Load data

In [9]:
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    # convert lists to np.array e.g 'mapping', 'labels' in json file
    inputs = np.array(data["mfcc"])[..., np.newaxis] # 4d array --> (num_samples, 130, 13, 1)
    targets = np.array(data["labels"])
    
    return inputs, targets

Test loading

In [10]:
inputs, targets = load_data(JSON_PATH)

In [11]:
print(inputs.shape)
print(targets.shape)

(8996, 130, 13)
(8996,)


Data labels

In [16]:
for semantic_label, target_label in zip(semantic, np.unique(targets)):
    print(f"Target label for {semantic_label}: {target_label}")

Target label for blues: 0
Target label for classical: 1
Target label for country: 2
Target label for disco: 3
Target label for hiphop: 4
Target label for metal: 5
Target label for pop: 6
Target label for reggae: 7
Target label for rock: 8


### Train / Test Split

A simple `train_test_split` can be performed on the data for the baseline multilayer peceptron model. 

The following function would be used to split data into `train | test | validation` for the CNN model:

In [17]:
def split_data(test_size= 0.25, validation_size= 0.2, dataset_path= JSON_PATH_MAIN, random_state=42):
    """split train data into train, vaidation and test sets"""
    # load_data
    X, y = load_data(dataset_path)
    
    # train / test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state= random_state)
    
    # train / val split
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state= random_state)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [20]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data()